In [ ]:
GITRepoDirectory = "/Users/yangruiling/Desktop/iftbigdatacoursework2019/RuilingYang/DMOBondPrices"

In [ ]:
pip install Flask-SQLAlchemy

In [ ]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
from datetime import datetime 
from pandas import Series
register_matplotlib_converters()

In [ ]:
engine = create_engine(f"sqlite:///{GITRepoDirectory}/BondPrice.db")

In [ ]:
con = engine.connect()

In [ ]:
def BondStatic():
    return ("SELECT * FROM bond_static")

BondStatic = pd.read_sql_query(BondStatic(),engine)

In [ ]:
BondStatic

In [ ]:
BondStatic.index

In [ ]:
print(BondStatic["issue_date"])

In [ ]:
def BondPrices():
    return ("SELECT * FROM bond_prices")
BondPrices = pd.read_sql_query(BondPrices(),engine)

In [ ]:
type(BondPrices)

In [ ]:
BondPrices

In [ ]:
BondPrices.index

In [ ]:
BondPrices['cob_date'] = pd.to_datetime(BondPrices['cob_date'])
BondPrices.set_index("cob_date", inplace=True)

In [ ]:
BondPrices.index

In [ ]:
BondPrices.head()

In [ ]:
len(BondPrices.groupby('isin_id'))

In [ ]:
mean_clean=BondPrices.groupby("isin_id").apply(lambda x: np.mean(x.clean_price))
var_clean=BondPrices.groupby('isin_id').apply(lambda x: np.var(x.clean_price))

In [ ]:
mean_dirty=BondPrices.groupby('isin_id').apply(lambda x: np.mean(x.dirty_price))
var_dirty=BondPrices.groupby('isin_id').apply(lambda x: np.var(x.dirty_price))

In [ ]:
mean_ai=BondPrices.groupby('isin_id').apply(lambda x: np.mean(x.acc_int))
var_ai=BondPrices.groupby('isin_id').apply(lambda x: np.var(x.acc_int))

In [ ]:
num_by_id = (BondPrices.isin_id.value_counts()).sort_index()
statistic = pd.DataFrame(num_by_id)

In [ ]:
statistic['var_clean']=var_clean
statistic['mean_clean']=mean_clean
statistic['var_dirty']=var_dirty
statistic['mean_dirty']=mean_dirty
statistic['var_ai']=var_ai
statistic['mean_ai']=mean_ai
statistic

In [ ]:
statistic[statistic['var_clean'].isin([max(var_clean)])]

In [ ]:
max_clean_bond=pd.DataFrame(BondPrices[BondPrices['isin_id'].isin(['GB00BDX8CX86'])])
%config InlineBackend.figure_format = 'retina'
fig, ax = plt.subplots()
_ = ax.set_title ("Clean price of GB00BDX8CX86")
_ = ax.plot(max_clean_bond['clean_price'], marker='o', markersize=1)
_ = ax.set_ylabel("Clean price")
_ = ax.set_xlabel('time')

In [ ]:
statistic[statistic['var_dirty'].isin([max(var_dirty)])]

In [ ]:
max_dirty_bond=pd.DataFrame(BondPrices[BondPrices['isin_id'].isin(['GB00B0CNHZ09'])])
%config InlineBackend.figure_format = 'retina'
fig, ax = plt.subplots()
_ = ax.set_title ("Dirty price of GB00B0CNHZ09")
_ = ax.plot(max_dirty_bond['dirty_price'], marker='o', markersize=1)
_ = ax.set_ylabel("Dirty price")
_ = ax.set_xlabel('time')

In [ ]:
statistic[statistic['mean_clean'].isin([max(mean_clean)])]

In [ ]:
statistic[statistic['mean_dirty'].isin([max(mean_dirty)])]

In [ ]:
max_dirty_bond=pd.DataFrame(BondPrices[BondPrices['isin_id'].isin(['GB0009081828'])])
%config InlineBackend.figure_format = 'retina'
fig, ax = plt.subplots()
_ = ax.set_title ("Dirty price of GB0009081828")
_ = ax.plot(max_dirty_bond['dirty_price'], marker='*', markersize=1)
_ = ax.set_ylabel("Dirty price")
_ = ax.set_xlabel('time')

In [ ]:
def MaxDirtyBond():
    return ("SELECT * FROM bond_prices WHERE isin_id = 'GB0009081828'")
MaxDirtyBond = pd.read_sql_query(MaxDirtyBond(),engine)

In [ ]:
MaxDirtyBond

In [ ]:
MaxDirtyBond.clean_price.describe()

In [ ]:
MaxDirtyBond.dirty_price.describe()

In [ ]:
MaxDirtyBond.index

In [ ]:
MaxDirtyBond['clean_return'] = np.nan
MaxDirtyBond['dirty_return'] = np.nan
MaxDirtyBond['dirty_SD'] = np.nan
MaxDirtyBond['dirty_AVG'] = np.nan


In [ ]:
for a in range(1,len(MaxDirtyBond.clean_price)):
    MaxDirtyBond.loc[a,'clean_return'] = np.log(MaxDirtyBond.loc[a,'clean_price'])-np.log(MaxDirtyBond.loc[a-1,'clean_price'])
    MaxDirtyBond.loc[a,'dirty_return'] = np.log(MaxDirtyBond.loc[a,'dirty_price'])-np.log(MaxDirtyBond.loc[a-1,'dirty_price'])
    
    if a > 9 :
        b = a - 10
        MaxDirtyBond.loc[a,'dirty_SD'] = MaxDirtyBond.loc[b:a,'dirty_return'].std()
        MaxDirtyBond.loc[a,'dirty_AVG'] = MaxDirtyBond.loc[b:a,'dirty_return'].mean()

In [ ]:
MaxDirtyBond.head(50)

In [ ]:
%config InlineBackend.figure_format = 'retina'
fig, ax = plt.subplots()
_ = ax.set_title ("Clean Price Return of GB0009081828")
_ = ax.plot(MaxDirtyBond['clean_return'], marker='o', markersize=1)
_ = ax.set_ylabel("Clean Price Return")
_ = ax.set_xlabel('Days')

In [ ]:
MaxDirtyBond.clean_return.describe()

In [ ]:
%config InlineBackend.figure_format = 'retina'
fig, ax = plt.subplots()
_ = ax.set_title ("Dirty Price Return of GB0009081828")
_ = ax.plot(MaxDirtyBond['dirty_return'], marker='o', markersize=1)
_ = ax.set_ylabel("Dirty Price Return")
_ = ax.set_xlabel('Days')

In [ ]:
MaxDirtyBond.dirty_return.describe()